In [15]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from semantic_kernel.contents.chat_history import ChatHistory  
from dotenv import load_dotenv
from os import environ
from tracing import set_up_all


from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

load_dotenv(override=True)




True

In [ ]:
## SEMANTICKERNEL_EXPERIMENTAL_GENAI_ENABLE_OTEL_DIAGNOSTICS_SENSITIVE=true
set_up_all(connection_string=environ.get("AZURE_INSIGHT_CONNECTION_STRING"))

In [3]:
kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))



In [4]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.
- Return customer_id and product_id alongside names in all outputs.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
"""

In [5]:

sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()

agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgent", 
    instructions=instructions,
    plugins=[sales_plugin, ]
)


In [6]:
messages = [
    "Which is the revenue for Brake_Pads?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]


In [7]:

response = await agent.get_response(messages=messages[0])
print(response)


The total revenue generated from sales of the "Brake Pad" product is **$15,406.0 USD**. This value is calculated from the sum of the line unit prices across all orders for "Brake Pad".


In [8]:
response2 = await agent.get_response(messages=messages[1], thread=response.thread)
print(response2)



### Customer Details
1. **Customer Name**: AutoZone  
   - **Customer ID**: 2  
   - **Region**: NA  
   - **Industry**: Distributor  
   - **Account Manager**: Carol Lee  

2. **Customer Name**: Bosch  
   - **Customer ID**: 3  
   - **Region**: EU  
   - **Industry**: OEM  
   - **Account Manager**: David Wong  

3. **Customer Name**: Ford  
   - **Customer ID**: 0  
   - **Region**: NA  
   - **Industry**: OEM  
   - **Account Manager**: Alice Johnson  

4. **Customer Name**: GM  
   - **Customer ID**: 1  
   - **Region**: NA  
   - **Industry**: OEM  
   - **Account Manager**: Bob Smith  

5. **Customer Name**: NAPA  
   - **Customer ID**: 4  
   - **Region**: NA  
   - **Industry**: Distributor  
   - **Account Manager**: Ellen Garcia  

---

### Product Details
1. **Product Name**: Alternator  
   - **Product ID**: 3  
   - **Category**: Electrical  
   - **Unit Cost**: $90.0 USD  
   - **Unit Price**: $200.0 USD  

2. **Product Name**: Brake Pad  
   - **Product ID**: 0  
   - *

In [9]:
response3 = await agent.get_response(messages=messages[2], thread=response.thread)
print(response3)

The customer with the highest sales for "Brake Pad" is:

### Customer Details:
- **Customer Name**: GM
- **Customer ID**: 1
- **Total Line Revenue**: $4,950.0 USD
